In [12]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "../src"))

In [13]:
import logging
import jax
import jax.numpy as np
import numpy as onp
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import statsmodels.api as sm
from datetime import datetime

from simulate_data_fct import sim_IV_ilr_linear, sim_IV_ilr_nonlinear
from plot_fct import update_layout, img_path, update_layout_px, colours, plot_ilr_X_vs_Y
from run_methods_all import run_methods_confidence_interval
from plot_fct import plot_mse_results, plot_beta_results

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# set up logging file
logging.getLogger().setLevel(logging.INFO)

In [16]:
key = jax.random.PRNGKey(253)

# Linear Setup

## Data Generation

In [17]:
n = 1000
p = 3
V = cmp._gram_schmidt_basis(p)
instrument_strength = "weak"
savepath = os.path.join(img_path, "LinearSetUp")
num_inst = 2
c_X = np.array([1, 1])  # confounder multiplication to X
alpha0 = np.array([4, 1])  # INSTRUMENT intercept
alphaT = np.array([[+0.15, 0.15],  # INSTRUMENT INTERCEPT FIrst Component of Microbiome
                    [0.2, 0]])  # INSTRUMENT INTERCEPT Second Component of Microbiome
beta0 = 2
betaT = np.array([6, 2])
betaT_log = V.T@betaT
mu_c = -2
c_Y = 4  

In [18]:
confounder, Z_sim, X_sim, Y_sim, X_star, Y_star = sim_IV_ilr_linear(
            key,
            n=n,
            p=p,
            num_inst=num_inst,
            mu_c=mu_c,
            c_X=c_X,
            alpha0=alpha0,
            alphaT=alphaT,
            c_Y=c_Y,
            beta0=beta0,
            betaT=betaT,
            num_star=500)
X_sim_ilr = cmp.ilr(X_sim)
X_star_ilr = cmp.ilr(X_star)

## Data Visualization

In [19]:
fig = px.bar(pd.DataFrame(X_sim[:100, :], columns=["Microbe 1", "Microbe 2", "Microbe 3"]))
fig = update_layout_px(fig)
fig.update_xaxes(title="Sample number")
fig.update_yaxes(title="Percentage of microbiota in sample")
fig.write_image(os.path.join(savepath,
                             "Barplot_Sample_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [20]:
# Compute F-Statistics 
ZZ_sim = onp.array(sm.add_constant(Z_sim))
alphahat = np.linalg.inv(ZZ_sim.T@ZZ_sim)@ZZ_sim.T@X_sim_ilr
MSM = np.sum(((ZZ_sim@alphahat - np.mean(X_sim_ilr, axis=0))**2), axis=0)/(num_inst)
MSE = np.sum(((ZZ_sim@alphahat - X_sim_ilr)**2), axis=0)/(n-num_inst)
F = MSM / MSE
F

DeviceArray([1.109971  , 0.69842106], dtype=float32)

In [21]:
df = pd.DataFrame(X_sim, columns=["Microbe 1", "Microbe 2", "Microbe 3"])
fig = px.scatter_ternary(df, a='Microbe 1', 
                         b='Microbe 2', 
                         c='Microbe 3',
                         color=Z_sim[:, 0].squeeze(),
                        labels={"color": "Instrument value"})

fig = update_layout_px(fig)
#ig.update_layout(gridcolor="gray")
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ1_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [22]:
fig = px.scatter_ternary(df, a='Microbe 1', 
                         b='Microbe 2', 
                         c='Microbe 3',
                         color=Z_sim[:, 1].squeeze(),
                        labels={"color": "Instrument value"})

fig = update_layout_px(fig)
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ2_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

In [63]:
fig = plot_ilr_X_vs_Y(X_sim_ilr, X_star_ilr, Y_sim, Y_star)
fig.update_layout(width=2000, height=500)
fig.write_image(os.path.join(savepath,
                             "Data_vs_trueeffect_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

# Diversity Approach

In [24]:
from plot_fct import plot_diversity_methods
from helper_fct import diversity
from run_methods_all import run_diversity_estimation_methods
div_shannon = diversity(X_sim, "shannon")
div_simpson = diversity(X_sim, "simpson")

In [25]:
# Shannon Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_shannon, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

fig = plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "ShannonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                   0.02492
Date:                 Fr, 11 Jun 2021   Prob (F-statistic):              0.975
Time:                        10:21:43   Log-Likelihood:                -1418.9
No. Observations:                1000   AIC:                             2844.
Df Residuals:                     997   BIC:                             2859.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0022      0.084     -0.026      0.9

INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [-2.29989822]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [6.49240054]...
INFO:absl:Predict treatment effect...


In [26]:
# Simpson Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_simpson, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "SimpsonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...
INFO:absl:Optimize lambda...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                   0.08363
Date:                 Fr, 11 Jun 2021   Prob (F-statistic):              0.920
Time:                        10:21:46   Log-Likelihood:                -1418.9
No. Observations:                1000   AIC:                             2844.
Df Residuals:                     997   BIC:                             2858.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0153      0.084     -0.182      0.8

INFO:absl:Optimal lambda [-2.42124119]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [6.27582759]...
INFO:absl:Predict treatment effect...


# Confidence Intervals for higher dimensional methods

In [27]:
from run_methods_all import run_methods_all

In [28]:
today = datetime.now()
num_iteration = 50

path = os.getcwd()
mypath = os.path.join(path, "temp",
                      today.strftime("%Y_%m_%d_%H_%M") + "_LINEAR_ConfidenceInterval_"+str(instrument_strength)
                      +"_"+ str(num_iteration)+"_Microbiota_" +str(p))

if not os.path.isdir(mypath):
    os.makedirs(mypath)


# create logging file with relevant information
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=os.path.join(os.getcwd(), mypath, "configurations.log"),
                    filemode='w'
                    )

logging.info("n="+str(n)+", p="+str(p)+", instrument strength: "+instrument_strength)
logging.info("Number of runs: "+str(num_iteration))
logging.info("First Stage Specification: ")
logging.info("Number of Instruments: "+str(num_inst))
logging.info("Instrument Strength by F-Test: " +str(F))
logging.info("alpha0="+str(alpha0)+", alphaT="+str(alphaT)+", confounder multiplicator c_X="+str(c_X))
logging.info("Second Stage Specification: ")
logging.info("beta0="+str(beta0)+", betaT="+str(betaT)+", confounder multiplicator c_Y="+str(c_Y))
logging.info("Confounder mean mu_c="+str(mu_c))


df_beta, df_mse, mse_large_confidence= run_methods_confidence_interval(
    key, num_iteration, 
    n, p, num_inst, mu_c, c_X, alpha0, alphaT, c_Y, beta0, betaT, num_star=200)

**************************************************************************************************
*****************************************We are at 0 of 50***********************************************
**************************************************************************************************


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [7.44 4.54]
Estimated Beta: [ 7.12 -3.41 -3.71]
Error: 15.78

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
True Beta: [6 2]
Estimated Beta: [ 7.12 -3.41 -3.71]
Error: 15.78

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
Beta DirichLetLogContrast: -40.96[ 37.953 -30.181  -7.772]
True Beta: [6 2]
Estimated Beta: [48.18  9.52]
Error: 5054.68

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [7.69 4.29]
Estimated Beta: [ 7.19 -3.68 -3.5 ]
Error: 15.29

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
True Beta: [6 2]
Estimated Beta: [ 7.19 -3.68 -3.5 ]
Error: 15.3

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Error in computation
Beta DirichLetLogContrast: 1.497[10.118 -0.522 -9.596]
True Beta: [6 2]
Estimated Beta: [ 7.52 11.75]
Error: 126.39

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<

...

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

<Figure size 800x240 with 0 Axes>

In [29]:
import pickle
results = {
    "df_mse": df_mse,
    "df_beta": df_beta,
    "mse_large": mse_large_confidence
}

with open(os.path.join(savepath, "results_"+str(p)+str(instrument_strength)+".pickle"), "wb") as f:
    pickle.dump(results, f)
    f.close()


In [48]:
with open(os.path.join(savepath, "results_"+str(p)+str(instrument_strength)+".pickle"), "rb") as f:
    results = pickle.load(f)
    df_mse = results["df_mse"]
    df_beta = results["df_beta"]
    f.close()

# Mean Squarred Error Plot

In [57]:
filter_list = ["ALR+LC", "ILR+LC", "ILR+ILR", "DIR+LC",  "ONLY Second LC", "ONLY Second ILR"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
fig.update_yaxes(range=(0, 9500))
fig.write_image(os.path.join(savepath,
                             "MSE_Full_Linear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [61]:
filter_list = ["ALR+LC", "ILR+LC", "ILR+ILR", "ONLY Second LC", "ONLY Second ILR"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
fig.update_yaxes(range=(0, 25))
fig.write_image(os.path.join(savepath,
                             "MSE_Full_Linear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+"_2.pdf" ))
fig.show()

In [45]:
filter_list = ["ALR+LC", "DIR+LC", "ONLY Second LC"]

fig = plot_mse_results(df_mse, filter_list, sort_to_filter=True)
fig.update_yaxes(range=(0, 200))
fig.write_image(os.path.join(savepath,
                             "MSE_Small_Linear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

# Beta Plot

In [46]:
filter_list = ["ALR+LC", "DIR+LC", "ONLY Second LC"]
fig = plot_beta_results(df_beta, V.T@betaT, filter_list, sort_to_filter=True)
fig.update_layout(showlegend=False)
fig.update_yaxes(range=(-25, 25))
fig.write_image(os.path.join(savepath,
                             "Beta_Small_Linear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

In [53]:
filter_list = ["ALR+LC", "ILR+LC", "ILR+ILR", "DIR+LC",  "ONLY Second LC", "ONLY Second ILR"]
fig = plot_beta_results(df_beta, V.T@betaT, filter_list)
#fig.update_layout(legend=dict(
#    yanchor="top",
#    y=0.99,
#    xanchor="left",
#    x=0.01
#))
fig.update_layout(showlegend=False)
fig.update_layout(width=2000, height=500)
fig.update_yaxes(range=(-20, 40))
fig.write_image(os.path.join(savepath,
                             "Beta_Full_Linear" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()


In [105]:
filter_list = ["ALR+LC", "ILR+ILR", "ONLY Second ILR", "ONLY Second LC", "ILR+LC"]
fig = plot_beta_results(df_beta, V.T@betaT, filter_list)
#fig.update_layout(legend=dict(
#    yanchor="top",
#    y=0.99,
#    xanchor="left",
#    x=0.01
#))

fig.update_yaxes(range=(-15, 10))
fig.update_layout(width=2000, height=500)
fig.write_image(os.path.join(savepath,
                             "BetaFull2_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()
